# Закон Кюри-Вейсса

### Оценим допустимую ЭДС

In [11]:
delta_T = 0.5
k = 24
E = delta_T/k
print(E, "мВ")

0.020833333333333332 мВ


## Исследуем зависимость периода колебаний $\tau$ от температуры $T$

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import numpy as np
from sklearn import linear_model, metrics

In [32]:
df = pd.read_csv('main.csv')
df['T'] = df.T_m + k*df.U
df

,t,T_s,T_m,U,T
0,10.788,14.00,14.5,-0.005,14.380
1,10.725,15.83,16.3,-0.015,15.940
2,10.573,17.87,18.6,-0.015,18.240
3,10.288,20.02,20.6,-0.014,20.264
4,9.960,21.96,22.8,-0.016,22.416
5,9.610,23.96,24.8,-0.018,24.368
6,9.428,25.95,26.9,-0.018,26.468
7,9.340,27.96,28.6,-0.017,28.192
8,9.288,29.96,30.6,-0.017,30.192
9,9.252,31.96,32.6,-0.017,32.192


In [33]:
# TODO: Поменять на экспериментальное
t0 = 9.05

In [34]:
X = np.divide(1, np.power(df.t, 2) - np.power(t0, 2))
Y = df['T']

In [35]:
# Строим регрессию
regressor = linear_model.LinearRegression()
regressor.fit(X.values.reshape(-1, 1), Y.values.reshape(-1, 1))

f, ax = plt.subplots(figsize=(7, 5))
ax.scatter(X, Y, label='Exper. data', c='r')

X_pred = X.append(pd.Series([0]))[(X >= 0.05) | X == 0]
Y_pred = regressor.predict(X_pred.values.reshape(-1, 1))[X.index]

ax.plot(X_pred, Y_pred, label='Linear regression', c='black')
ax.set_title(r'Зависимость $T$ от $\frac{1}{(\tau^2 - \tau_0^2)}$', y=1.02)
ax.set_xlabel(r'$\frac{1}{\tau^2-\tau_0^2}$, $\left[ \mu s^{-2} \right]$')
ax.set_ylabel(r'$T, [K]$')

ax.yaxis.set_minor_locator(MultipleLocator(1))
ax.xaxis.set_minor_locator(MultipleLocator(0.025))
ax.set_xlim(0, 0.4)
#ax.set_yticks(np.append(ax.get_yticks(), [np.round(regressor.intercept_[0], 2)]))
ax.annotate(round(regressor.intercept_[0], 2), 
            xy=(0, regressor.intercept_[0]), 
            xytext=(-0.05, regressor.intercept_[0]),
           arrowprops={"facecolor": "black", "arrowstyle":"->",})
ax.grid(True, which='minor', linestyle='--')
ax.grid(True, which='major', c='black', linestyle='-')

f.text(x=0.15, y=0.72, s=r"$\theta _p = {}$".format(round(regressor.intercept_[0], 2)), bbox={"alpha": 1, "facecolor": "white"})
ax.legend()

plt.savefig('out.pdf', fmt='pdf')
plt.show(f)

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.

In [20]:
print("Уголовой коэффициент: {} \nТочка пересечения (константный признак): {}\nСтандартное отклонение: {}".format(regressor.coef_[0][0], regressor.intercept_[0], metrics.mean_squared_error(Y, Y_pred)))

ValueError: Found input variables with inconsistent numbers of samples: [10, 11]